In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from hansolo import *
import psutil
from sklearn import datasets
from sklearn.model_selection import train_test_split
import time

#retrieve the dataset
digits = datasets.load_digits()
images=digits.images/16 #(1797,8,8)
target=digits.target #(1797,)



X_train, X_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.2, random_state=42)
M_train = np.shape(X_train)[0]
M_test = np.shape(X_test)[0]

# Reshape images and compute probabilities
flattened_images_train = X_train.reshape((M_train, -1))
flattened_images_test = X_test.reshape((M_test, -1))

#network parameters
M0 = 40 #number of images to train hidden layer
M0_sel = 40 #number of images to select hidden neurons
M_out = M_train - M0 - M0_sel #number of images to train output layer
M_tot = M0 + M0_sel + M_out + M_test

N = 2000 #number of time steps
p = 1  # spiking proba input neurons if pixel is black
nu = 0.5 # bias


K_input = 64 #number of input neurons

K_mid = int(K_input * (K_input - 1)/2) #number of hidden neurons before selection
#K_mid_new = 120

K_output = 10 #number of output neurons
eta_mid = 3 #parameter EWA for hidden neurons
expert1 = 'PWA' #expert of hidden layer
para_PWA = 2 #parameter PWA 

expert2 = 'PWA' #expert of output layer

indices = np.arange(K_input)
i, j = np.meshgrid(indices, indices)
mask = i < j
l_mid = np.column_stack((i[mask], j[mask])) #tensor indices of hidden neurons

eta_output = 0.002 #parameter EWA for output neurons

images_mid = flattened_images_train[:M0+M0_sel,:] /16
images_out = flattened_images_train[M0+M0_sel:M0+M0_sel+M_out,:] /16
images_test = flattened_images_test /16

target_out = y_train[M_train - M_out:]
target_test = y_test
Nb_simu = 20 #number of simulations

accuracy = np.zeros((Nb_simu))

for K_mid_new in np.arange(10,210,10): #number of selected neurons hidden layer
    for nb in range(Nb_simu):
    #Simulation input neurons
        prob =  np.transpose(np.array([p*images_mid for i in range(N)]), axes = (2,0,1))
    # Input Neurons activity (K_input,N,M)

        input_neurons_mid = stats.bernoulli.rvs(prob) #to train mid neurons

    #to train output neurons
        prob_input = np.transpose(np.array([p*images_out for i in range(N)]), axes = (2,0,1))
        input_neurons_out = stats.bernoulli.rvs(prob_input)
        prob_input = np.transpose(np.array([p*images_test for i in range(N)]), axes = (2,0,1))
        input_neurons_test = stats.bernoulli.rvs(prob_input) #to test
    #Parameters mid neurons
        W_mid = np.zeros((K_mid,K_input,M0))
        W_mid[:,:,0] = 1/K_input
        W_mid_not_renorm = np.zeros((K_mid, K_input, M0))
        W_mid_not_renorm[:, :, 0] += 1

        cum_gain_mid = np.zeros((K_mid))
        cum_gain_input_to_mid = np.zeros((K_mid,K_input))

    #training mid neurons

        for m in range(M0-1):
        
            gain_mid = GainMid(np.arange(K_mid),K_mid,K_input,input_neurons_mid[:,:,m], l_mid) #(K_mid,K_input)
            cum_gain_input_to_mid += gain_mid
            cum_gain_mid += np.sum(W_mid[:,:,m] * gain_mid, axis = 1)

            if expert1 == "EWA" and m < M0-1:
                W_mid_not_renorm[:, :, m + 1] = EWA(W_mid_not_renorm[:, :, m], eta_mid, gain_mid)
                W_mid[:, :, m + 1] = (W_mid_not_renorm[:, :, m + 1].T / np.sum(W_mid_not_renorm[:, :, m + 1], axis=1)).T
            #W_mid[:, :, m+1] = 1/K_mid
                
            else:
                W_mid_not_renorm[:, :, m + 1] = PWA(para_PWA, K_mid, K_input, cum_gain_mid, cum_gain_input_to_mid) 
                not_zero = np.where(np.sum(W_mid_not_renorm[:,:,m+1], axis = 1) != 0)[0]
                zero = np.where(np.sum(W_mid_not_renorm[:,:,m+1], axis = 1) == 0)[0]
                W_mid[not_zero, :, m + 1] = (W_mid_not_renorm[not_zero, :, m + 1].T / np.sum(W_mid_not_renorm[not_zero, :, m + 1], axis=1)).T
                W_mid[zero, :, m+1] = 1/K_input

        #print(nb,"mid neurons trained")

    #selection mid neurons

        prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid[:,:,M0-1], input_neurons_mid[:, :, M0:M0 + M0_sel])
        clipped_probas_mid = np.clip(prob_mid, 0, 1)
        mid_neurons_sel = stats.bernoulli.rvs(clipped_probas_mid)
        P_mid = np.sum(mid_neurons_sel, axis=1) / N
        max_P = np.max(P_mid, axis = 1 )
        max_P_sorted = np.sort(max_P)

        x = max_P_sorted[K_mid-K_mid_new] 
        mid_neur_ok = np.where(max_P > x)[0] #selected neurons
        missing = K_mid_new - np.shape(mid_neur_ok)[0]
        mid_neur_to_sel = np.where(max_P == x)[0]
        n_tot = np.shape(mid_neur_to_sel)[0]
        selected_indices = np.random.choice(n_tot, size=missing, replace=False) #random choice of neurons which have the same nb of spikes
        mid_neur_missing = mid_neur_to_sel[selected_indices]

        mid_neur = np.concatenate((mid_neur_missing, mid_neur_ok))
        mid_neur = np.sort(mid_neur)


        prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid[mid_neur,:,M0-1], input_neurons_out)
        clipped_probas_mid = np.clip(prob_mid, 0, 1)
        mid_neurons_out=stats.bernoulli.rvs(clipped_probas_mid)

        P_mid = np.sum(mid_neurons_out, axis = 1) / N

    #parameters output
    
        cum_gain_output = np.zeros((K_output))
        cum_gain_mid2_to_output = np.zeros((K_output, K_mid_new))

        W_output = np.zeros((K_output, K_mid_new, M_out))
        W_output_not_renorm = np.zeros((K_output, K_mid_new, M_out))
        W_output[:, :, 0] += 1/K_mid_new
        W_output_not_renorm[:, :, 0] += 1

    #training output

        for m in range(M_out-1):
            gain_output = GainOutput(K_output, K_mid_new, P_mid[:,m], target_out[m]) 
            cum_gain_mid2_to_output += gain_output
            cum_gain_output += np.sum(W_output[:, :, m] * gain_output, axis = 1)
            

            if expert2 == "EWA":
                W_output_not_renorm[ :, :, m+1] = EWA(W_output_not_renorm[:, :, m], eta_output, gain_output)
                W_output[:,:, m+1] = (W_output_not_renorm[:, :, m+1].T / np.sum(W_output_not_renorm[:, :, m + 1], axis = 1)).T

            else:
                W_output_not_renorm[:, :, m+1] = PWA(para_PWA, K_output, K_mid_new, cum_gain_output, cum_gain_mid2_to_output)
                
                not_zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) != 0)[0]
                zero = np.where(np.sum(W_output_not_renorm[:,:,m+1], axis = 1) == 0)[0]
                W_output[not_zero,:, m + 1] = (
                    W_output_not_renorm[not_zero, :, m + 1].T / np.sum(W_output_not_renorm[not_zero,:, m + 1], axis=1)
                    ).T
                W_output[zero,:, m+1] = 1/K_mid_new

        #print(nb, 'output neurons trained')

    #test with new images
        prob_mid = -nu + np.einsum('ai, itm -> atm', W_mid[mid_neur,:,M0-1], input_neurons_test)
        clipped_probas_mid = np.clip(prob_mid, 0, 1)
        mid_neurons_test=stats.bernoulli.rvs(clipped_probas_mid)

        answers_test = np.zeros((M_test))
        for m in range(M_test):
        
        # Simulation output neurons (without Kalikow)
            probas_output = np.sum(W_output[:,None,:,- 1] * mid_neurons_test[:,:,m].T, axis = 2)
            probas_output = np.clip(probas_output, 0, 1)
            output_neurons = stats.bernoulli.rvs(probas_output)

            P_output = np.sum(output_neurons, axis = 1) / N
            chosen_class = np.where(P_output == np.max(P_output))[0]

            if chosen_class[0] == target_test[m]:
                answers_test[m] = 1
        accuracy[nb] = np.sum(answers_test)/M_test

    #np.save(f"accuracy_{expert1}_{expert2}_{K_mid_new}_{eta_output}_20", accuracy)



